In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('final_data1.csv')

In [3]:
movie_sth = pd.read_csv('/data/data/Movie.txt',sep='\t',encoding='utf-8')

In [4]:
all_movie_info = pd.merge(data,movie_sth,on=['mid'],how='left')

In [5]:
for i in all_movie_info.select_dtypes('object').columns:
    lbl = LabelEncoder()
    all_movie_info[i] = lbl.fit_transform(all_movie_info[i].fillna(-1).astype(str))

In [6]:
from sklearn.cross_validation import train_test_split
feature_name = [i for i in all_movie_info if i not in ['star']]
train_data,valid_data,train_label,valid_label = train_test_split(all_movie_info[feature_name],all_movie_info['star'],random_state=1998,test_size=0.2)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
import lightgbm as lgb
dtrain = lgb.Dataset(train_data[feature_name], label=train_label.values)
dval = lgb.Dataset(valid_data[feature_name], label=valid_label.values)

params = {'learning_rate': 0.1,
          'metric': ['auc','binary_logloss'],
          'objective': 'binary',
          'nthread': 32,
          'num_leaves': 16,
          'colsample_bytree': 0.9,
          'bagging_fraction' : 0.9,
          'bagging_freq' : 10,
          'seed' : 2018,
        }
         
lgb_model = lgb.train(params, dtrain, 1000, dval, verbose_eval=10,early_stopping_rounds=100,)

Training until validation scores don't improve for 100 rounds.
[10]	valid_0's binary_logloss: 0.285269	valid_0's auc: 0.957123
[20]	valid_0's binary_logloss: 0.218227	valid_0's auc: 0.967723
[30]	valid_0's binary_logloss: 0.195773	valid_0's auc: 0.970419
[40]	valid_0's binary_logloss: 0.185643	valid_0's auc: 0.972144
[50]	valid_0's binary_logloss: 0.178916	valid_0's auc: 0.97382
[60]	valid_0's binary_logloss: 0.173798	valid_0's auc: 0.975147
[70]	valid_0's binary_logloss: 0.171597	valid_0's auc: 0.975678
[80]	valid_0's binary_logloss: 0.168935	valid_0's auc: 0.976382
[90]	valid_0's binary_logloss: 0.167078	valid_0's auc: 0.976899
[110]	valid_0's binary_logloss: 0.164889	valid_0's auc: 0.977466
[120]	valid_0's binary_logloss: 0.164303	valid_0's auc: 0.977644
[130]	valid_0's binary_logloss: 0.163708	valid_0's auc: 0.977799
[140]	valid_0's binary_logloss: 0.16306	valid_0's auc: 0.977972
[150]	valid_0's binary_logloss: 0.162359	valid_0's auc: 0.97815
[160]	valid_0's binary_logloss: 0.16175

In [13]:
lgb_model.save_model('model-original.txt')

In [30]:
feature_name

['uid',
 'mid',
 'timeStamp',
 'name',
 'director',
 'scriptwriter',
 'mainPlayers',
 'categories',
 'productCountry',
 'duration',
 'releasedTime',
 'tags',
 'ratingCount',
 'star5',
 'star4',
 'star3',
 'star2',
 'star1',
 'language']